#NOME FIGO
Art and history of art are no sealed compartments: they are heavily inter-dependent with social, political, economic factors, which in turn influence our very perception of what art is.

Cultural institutions and museums in particular play a fundamental role in this intertwined dynamics: through their selection activity, they have the potential to shape the public understanding of arts and its modifications throughout time. 
In some way, what makes into museums makes into history of art.

From these considerations stems our analysis: how do external (social, political, economic) factors influence the perception of art and its history?
A way to investigate it is by looking at the greatest and most representative museums around the world, and at their acquisition policies and campaigns in particular.

Our key questions:
In which ways have the acquisition campaigns of the major museums in the world changed throughout the years? 


Our workflow:
1. Interrogate WikiData:
    - What are the biggest collections around the world?
2. Find csv files for some of the major museums.
3. Select some representative time slots (both internal and external factors).
4. Analyse acquisitions during these time slots for every museum and compare:
    a) Difference between different slots in the same museum;
    b) Difference between different museums for the same time slot;

Our questions:
- What was the initial nucleus of each museum? 
- Internal survey: Is there a significant date or decade for the acquisitions? 
- External survey: What are the acquisition trends around the Xs/between the x and the y? / What are the acquisition trends within and across these museums? 
- During these years, who are the most represented makers? What is the most represented gender? What is the most represented movement? What is the most represented nationality? 


We analysed 5/4 of the (MET, MoMa, N+, Cleveland?, Tate) 

Wikidata interrogation: failure.

1. What are the largest art collections?

SELECT ?museum (COUNT(?work) AS ?works) WHERE {
  ?work wdt:P195 ?museum.
  ?museum wdt:P31 wd:Q207694
  }
GROUP BY ?museum 
ORDER BY DESC(?works)

2.  Which were the most visited museums in 2018?

SELECT ?museumLabel ?visitors ?year
WHERE {
  ?museum wdt:P31 wd:Q207694;
          wdt:P1705 ?museumLabel;
          wdt:P1174 ?visitors;
          p:P1174/pq:P585 ?year .
FILTER(YEAR(?year) = 2018).
}

ORDER BY DESC(?visitors)

Since WikiData was not providing reliable results, we decided to go back to its sources (The Art Newspaper https://www.theartnewspaper.com/) and manually collect data about the most visited museums in the last four years(2018-2022).

https://onedrive.live.com/view.aspx?resid=E34DDE1A3F2F2160!138&ithint=file%2cxlsx&authkey=!AN4u-K4bko37iOU
    
We verified the availability of open datasets for each of the top 20 most visited museums on this GitHub repository (https://github.com/Ambrosiani/museums-on-github), containing a list of museums with GitHub accounts.

Our analysis led us to the decision to focus on four museums:
- Tate Modern, London
- MoMa, NY
- Met, NY
- National Gallery of Art, Washington DC

**Info generale sui musei.

In [2]:
import pandas as pd
import csv
import re

# Creating dataframes from online CSVs

First: let us create some pandas dataframes containing all needed information: for each Museum, we will integrate different csv files, selecting the data we need for each of them. 

## MoMa 

In [146]:
spreadsheet = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')
pd.set_option('display.max_columns', None)
artworks = spreadsheet[['Title', 'Artist', 'ConstituentID', 'Nationality', 'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'CreditLine', 'Classification', 'Department', 'DateAcquired', 'URL']]
artists = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artists.csv')
artists["ConstituentID"] = artists["ConstituentID"].astype(str)
MoMa = pd.merge(artworks,artists[['ConstituentID', 'Wiki QID']],on='ConstituentID', how='left')
MoMa.rename(columns = {'ConstituentID':'Id', 'BeginDate':'BirthDate', 'EndDate':'DeathDate'}, inplace = True)

## Tate

In [147]:
spreadsheet = pd.read_csv('https://raw.githubusercontent.com/tategallery/collection/master/artwork_data.csv')
pd.set_option('display.max_columns', None)
artworks = spreadsheet[['artist', 'artistId', 'title', 'medium', 'creditLine', 'year', 'acquisitionYear', 'url']]
artworks.rename(columns = {'artistId':'id'}, inplace = True)
artworks.id = artworks.id.astype(str)
artists = pd.read_csv('https://raw.githubusercontent.com/tategallery/collection/master/artist_data.csv')
artists["id"] = artists["id"].astype(str)
Tate = pd.merge(artworks,artists[['id', 'gender', 'yearOfBirth', 'yearOfDeath']], on='id', how='left')
Tate.rename(columns = {'artist':'Artist', 'id':'Id', 'title':'Title', 'yearOfBirth':'BirthDate', 'yearOfDeath':'DeathDate', 'medium':'Medium', 'creditLine':'CreditLine', 'year':'Date', 'acquisitionYear':'DateAcquired', 'url':'URL', 'gender':'Gender'}, inplace = True)

## Met

In [3]:
spreadsheet = pd.read_csv('https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv')
pd.set_option('display.max_columns', None)
Met = spreadsheet[['AccessionYear', 'Title', 'Culture', 'Artist Display Name', 'Artist Nationality', 'Artist Begin Date', 'Artist End Date', 'Artist Gender', 'Artist Wikidata URL', 'Object End Date', 'Medium', 'Credit Line', 'Classification', 'Link Resource', 'Object Wikidata URL']]
Met.rename(columns = {'Artist Display Name':'Artist', 'id':'Id', 'Artist Begin Date':'BirthDate', 'Artist End Date':'DeathDate', 'Credit Line':'CreditLine', 'Object End Date':'Date', 'AccessionYear':'DateAcquired', 'Artist Wikidata URL':'Wiki QID', 'Artist Gender':'Gender', 'Link Resource':'URL', 'Artist Nationality':'Nationality'}, inplace = True)

C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


## Nga

In [149]:
spreadsheet = pd.read_csv('https://raw.githubusercontent.com/NationalGalleryOfArt/opendata/main/data/objects.csv')
pd.set_option('display.max_columns', None)
Nga = spreadsheet[['accessionnum', 'title', 'endyear', 'medium', 'attribution', 'creditline', 'classification']]
Nga.rename(columns = {'attribution':'Artist', 'id':'Id', 'title':'Title', 'medium':'Medium', 'creditline':'CreditLine', 'endyear':'Date', 'accessionnum':'DateAcquired', 'classification':'Classification', 'Object End Date':'Date'}, inplace = True)

# Data cleaning

For each dataframe, we now want to do some cleaning

## Cleaning dates

In [150]:
def cleanDatesMoMa(date):
    if '-' in date:
        splitted = date.split('-')
        date = ' '.join(splitted) 
    if '/' in date:
        splitted = date.split('/')
        date = ' '.join(splitted) 
    if ',' in date:
        splitted = date.split(',')
        date = ' '.join(splitted) 
    if '.' in date:
        splitted = date.split('.')
        date = ' '.join(splitted) 
        
    x = re.search("\d{4}", date)
    if x:
        date = x.group()
    return date

In [7]:
def cleanDates(date):
    if '.' in date:
        date = date.split('.')[0]
    return date

### Cleaning artworks' creation and acquisition dates for each museum

In [11]:
Met.fillna('None', inplace=True)
Met["Date"] = Met["Date"].astype(str)
Met["Date"] = Met["Date"].apply(cleanDates)
Met["DateAcquired"] = Met["DateAcquired"].astype(str)
Met["DateAcquired"] = Met["DateAcquired"].apply(cleanDates)
Met.to_csv('Met.csv')

C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\FRANCE~1\AppData\Local\Temp/ipykernel_10260/2886862666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Met["Date"] = Met["Date"].astype(str)


NameError: name 'cleanDates' is not defined

In [5]:
MoMa.fillna(value='None', inplace=True)
MoMa["Date"] = MoMa["Date"].astype(str)
MoMa["Date"] = MoMa["Date"].apply(cleanDatesMoMa)
MoMa["DateAcquired"] = MoMa["DateAcquired"].astype(str)
MoMa["DateAcquired"] = MoMa["DateAcquired"].apply(cleanDatesMoMa)
MoMa.to_csv('MoMa.csv')


#values =['(n d )','TBD','nd','c  196?','no date','date of publicati','New York','Not available','Various','Various','unknown','Unknown','n d ','n d ','n d','n  d ','Unkown','TBC']
#Tate['Date']=Tate['Date'].replace(['(n d )','c','TBD','nd','c  196?','no date','date of publicati','New York','Not available','Various','Various','unknown','Unknown','n d ','n d ','n d','n  d ','Unkown','TBC'],value='None')
Tate.fillna(value='None', inplace=True)
Tate["Date"] = Tate["Date"].astype(str)
Tate["Date"] = Tate["Date"].apply(cleanDates)
Tate["DateAcquired"] = Tate["DateAcquired"].astype(str)
Tate["DateAcquired"] = Tate["DateAcquired"].apply(cleanDates)
Tate.to_csv('Tate.csv')

Met.fillna('None', inplace=True)
Met["Date"] = Met["Date"].astype(str)
Met["Date"] = Met["Date"].apply(cleanDates)
Met["DateAcquired"] = Met["DateAcquired"].astype(str)
Met["DateAcquired"] = Met["DateAcquired"].apply(cleanDates)
Met['Wiki QID'] = Met["Date"].astype(str)
Met.to_csv('Met.csv')


Nga.fillna(value='None', inplace=True)
Nga["Date"] = Nga["Date"].astype(str)
Nga["Date"] = Nga["Date"].apply(cleanDates)
Nga["DateAcquired"] = Nga["DateAcquired"].astype(str)
Nga["DateAcquired"] = Nga["DateAcquired"].apply(cleanDates)
Nga.to_csv('Nga.csv')

NameError: name 'MoMa' is not defined

# Exploring our Museums
<br>
Now that we have our dataframes, we can explore the four collections.
<br>
- How many items does each collection contain?
- Which timespan do items cover overall?
- First and last acquisition date for each museum. Tate's csv last update dates back to 2014.
- Total artists' number.
- Most represented artist, gender and nationality in general?

In [128]:
museums=[MoMa, Met, Tate, Nga]
names = ['Moma', 'Met', 'Tate', 'Nga']
for museum in museums:
    selected_rows = museum[~museum['Title'].isnull()]
    name = names.pop(0)
    print("Total artworks at", name, ":", len(selected_rows.index))

Total artworks at Moma : 140848
Total artworks at Met : 477804
Total artworks at Tate : 69201
Total artworks at Nga : 138037


# When do artworks date back?

Artworks' timespan <br>
For Met, we use a csv file we cleaned by hand.

In [158]:
museums=['Moma.csv','Nga.csv','Tate.csv', 'metclean2.csv']
names = ['Moma', 'Nga','Tate', 'Met']
for museum in museums:
    with open(museum, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        years=[]
        for item in reader:
            if item['Date'] != '' and item['Date'] != 'None'and item['Date'] != '(n d )'and item['Date'] != 'TBD'and item['Date'] != 'nd'and item['Date']!='c  196?' and 'c' not in item['Date'] and item['Date'] != 'no date' and item['Date'] != 'date of publicati' and item['Date'] != 'New York' and item['Date'] != 'Not available' and item['Date'] != 'Various' and item['Date'] != 'Various' and item['Date'] != 'unknown' and 'century' not in item['Date'] and item['Date'] != 'Unknown' and item['Date'] != 'n d ' and '(' not in item['Date'] and item['Date'] != 'n d ' and item['Date'] != 'n d' and item['Date'] != 'n  d ' and item['Date'] != 'Unkown' and item['Date'] != 'TBC':
                years.append(item['Date'])
        clean = []
        for el in years:
            clean.append(int(el))  
        clean.sort()
        name = names.pop(0)
    print("Most ancient artwork at", name, "dates back to", clean[0])
    print("Most recent artwork at", name, "dates back to", clean[-1])

Most ancient artwork at Moma dates back to 1768
Most recent artwork at Moma dates back to 2022
Most ancient artwork at Nga dates back to -490
Most recent artwork at Nga dates back to 2021
Most ancient artwork at Tate dates back to 1545
Most recent artwork at Tate dates back to 2012
Most ancient artwork at Met dates back to -240000
Most recent artwork at Met dates back to 2022


# When were artworks acquired?

## Acquisition' timespan

In [160]:
museums=['MoMa.csv','Met.csv', 'Nga.csv', 'Tate.csv']
names = ['MoMa','Met', 'Nga', 'Tate']
for museum in museums:
    with open(museum, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        acquisitionyears=[]
        for item in reader:
            if item['DateAcquired'] != 'None' and item['DateAcquired'] != 'Object Number':
                acquisitionyears.append(item['DateAcquired'])
        acquisitionyears.sort()
        name = names.pop(0)
    print("First recorderd acquisition", name, "dates back to", acquisitionyears[0])
    print("Last recorded acquisition", name, "dates back to", acquisitionyears[-1])

First recorderd acquisition MoMa dates back to 1929
Last recorded acquisition MoMa dates back to 2022
First recorderd acquisition Met dates back to 1870
Last recorded acquisition Met dates back to 2022
First recorderd acquisition Nga dates back to 1937
Last recorded acquisition Nga dates back to 2022
First recorderd acquisition Tate dates back to 1823
Last recorded acquisition Tate dates back to 2013


# Total artists' number.

In [14]:
def cleanArtistsTate(name):
    if ',' in name:
        splitted = name.split(',')
        name = ''.join(splitted) 
        return name

In [15]:
TateNew = Tate.copy(deep=True)
TateNew["Artist"] = TateNew["Artist"].apply(cleanArtistsTate)
TateNew.to_csv("TateNew.csv")

In [16]:
museums=['MoMa.csv', 'Met.csv', 'TateNew.csv', 'Nga.csv']
names = ['MoMa', 'Met','Tate', 'Nga']
for museum in museums:
    with open(museum, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        artists = set() 
        for item in reader:
            if item['Artist']!= '' and 'Unidentified'not in item['Artist'] and 'Various' not in item['Artist']:
                if ',' in item['Artist']:
                    item['Artist'] = item['Artist'].split(',')
                    for n in range(len(item['Artist'])):
                        artist= item['Artist'][n]
                        artists.add(artist)
                elif '|' in item['Artist']:
                    artists_list = item['Artist'].split('|')
                    for n in range(len(artists_list)):
                        artista= artists_list[n]
                        artists.add(artista)
                else:
                    artists.add(item['Artist'])
    name = names.pop(0)
    print("Number of artists at", name, "is", len(artists) )

Number of artists at MoMa is 14738
Number of artists at Met is 60950
Number of artists at Tate is 3281
Number of artists at Nga is 16860


Most represented gender in general?
Only for MoMa and Tate since other csv files need Wikidata integration

In [5]:
MoMaGender = MoMa.drop_duplicates(subset='Artist', keep="first")
MoMaGender.to_csv('MoMaGender.csv')

In [19]:
TateGender = Tate.drop_duplicates(subset='Artist', keep="first")
TateGender.to_csv('TateGender.csv')

In [20]:
museums=['MoMaGender.csv', 'TateGender.csv']
names = ['MoMa', 'Tate']
for museum in museums:
    with open(museum, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        Gender = {'Male':0, 'Female':0}
        for item in reader:
                if 'Male' in item['Gender']:
                    Gender['Male'] += 1
                if 'Female' in item['Gender']:
                    Gender['Female'] += 1
    print(Gender)

{'Male': 10474, 'Female': 2809}
{'Male': 2791, 'Female': 492}


Most represented nationality in general?

# Nationalities at MoMa

In [6]:
def cleanMoMaNationality(nationality):
    if ('(') or (')') in nationality:
        a = nationality.replace('(', '').replace(')', ',')
    return a.strip()
    

In [7]:
MoMaGender = MoMaGender[MoMaGender['Nationality'].notna()]
MoMaGender["Nationality"] = MoMaGender["Nationality"].apply(cleanMoMaNationality)
MoMaGender.to_csv('MoMaGender.csv')


In [8]:
museums=['MoMaGender.csv']
names = ['MoMa']
for museum in museums:
    with open(museum, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        nationalities = set() 
        for item in reader:
            if ',' in item['Nationality']:
                    item['Nationality'] = item['Nationality'].split(',')
                    for n in range(len(item['Nationality'])):
                        nationality= item['Nationality'][n].strip()
                        nationalities.add(nationality)
            else:
                    nationalities.add(item['Nationality'])
        count_naz= set()
        for el in nationalities:
            if el != '' and el != 'Nationality unknown' and el != ',' and el != ' ':
                count_naz.add(el)
                
print(count_naz)
print(len(count_naz))


{'Swedish', 'Venezuelan', 'Swiss', 'Icelandic', 'Canadian', 'Japanese', 'Kyrgyz', 'Slovak', 'Portuguese', 'Ghanaian', 'Costa Rican', 'Scottish', 'New Zealander', 'Greek', 'Algerian', 'Iranian', 'Ecuadorian', 'Irish', 'American', 'Brazilian', 'Vietnamese', 'Cambodian', 'Moroccan', 'Taiwanese', 'Zimbabwean', 'Norwegian', 'Kuwaiti', 'Sudanese', 'Malian', 'Uruguayan', 'Bolivian', 'Canadian Inuit', 'Ivorian', 'Tanzanian', 'Bulgarian', 'Puerto Rican', 'Finnish', 'Cypriot', 'Bahamian', 'Colombian', 'Indian', 'Congolese', 'Hungarian', 'Senegalese', 'Singaporean', 'Beninese', 'Chilean', 'Mozambican', 'Argentine', 'Malaysian', 'Namibian', 'Salvadoran', 'Czechoslovakian', 'Ugandan', 'Tunisian', 'Macedonian', 'Dutch', 'Palestinian', 'British', 'Thai', 'Catalan', 'Danish', 'South African', 'Albanian', 'Bosnian', 'Paraguayan', 'Yugoslav', 'Bangladeshi', 'Cameroonian', 'Panamanian', 'Nicaraguan', 'Luxembourger', 'Peruvian', 'Pakistani', 'Israeli', 'Lithuanian', 'Kenyan', 'Croatian', 'Azerbaijani', 'A

In [18]:
museums=['MoMaGender.csv']
names = ['MoMa']
for museum in museums:
    with open(museum, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        Nationalities ={}
        for nationality in count_naz:  
            Nationalities[nationality]= 0
        for item in reader:
            for naz in count_naz:
                    if naz in item['Nationality']:
                        Nationalities[naz] += 1 
    print(Nationalities)

{'Swedish': 123, 'Venezuelan': 68, 'Swiss': 421, 'Icelandic': 21, 'Canadian': 205, 'Japanese': 545, 'Kyrgyz': 1, 'Slovak': 9, 'Portuguese': 16, 'Ghanaian': 4, 'Costa Rican': 2, 'Scottish': 22, 'New Zealander': 10, 'Greek': 13, 'Algerian': 4, 'Iranian': 11, 'Ecuadorian': 4, 'Irish': 24, 'American': 5584, 'Brazilian': 185, 'Vietnamese': 3, 'Cambodian': 1, 'Moroccan': 8, 'Taiwanese': 4, 'Zimbabwean': 5, 'Norwegian': 35, 'Kuwaiti': 1, 'Sudanese': 2, 'Malian': 3, 'Uruguayan': 24, 'Bolivian': 3, 'Canadian Inuit': 3, 'Ivorian': 2, 'Tanzanian': 1, 'Bulgarian': 5, 'Puerto Rican': 6, 'Finnish': 52, 'Cypriot': 1, 'Bahamian': 1, 'Colombian': 57, 'Indian': 39, 'Congolese': 6, 'Hungarian': 86, 'Senegalese': 2, 'Singaporean': 2, 'Beninese': 1, 'Chilean': 68, 'Mozambican': 1, 'Argentine': 150, 'Malaysian': 2, 'Namibian': 2, 'Salvadoran': 2, 'Czechoslovakian': 3, 'Ugandan': 1, 'Tunisian': 2, 'Macedonian': 5, 'Dutch': 288, 'Palestinian': 3, 'British': 962, 'Thai': 7, 'Catalan': 1, 'Danish': 125, 'South 

# Nationalities at Met

In [45]:
def cleanNazMet(naz):
    if ',' in naz:
        naz = naz.split(',')[0]
    if '(' in naz:
        naz = naz.split('(')[0]
    if '?' in naz:
        naz = naz.replace('?', '')
    if '1866–1932' in naz:
        naz = naz.replace(' 1866–1932', '')
    if 'born' in naz:
        naz = naz.replace(' born', '')
    if ' and ' in naz:
        naz = naz.replace(' and ', '-') 
    if 'South ' in naz:
        naz= naz.replace('South ', '')
    if 'Southern ' in naz:
        naz= naz.replace('Southern ', '')
    if 'Northern ' in naz:
        naz= naz.replace('Northern ', '')
    if 'West ' in naz:
        naz= naz.replace('West ', '')
    if 'North ' in naz:
        naz= naz.replace('North ', '')
    if 'GErman' in naz:
        naz= naz.replace('GErman', 'German')
    if 'Geman' in naz:
        naz= naz.replace('Geman', 'German')
    if 'BRitish' in naz:
        naz= naz.replace('BRitish', 'British')
    if 'Britsh' in naz:
        naz= naz.replace('Britsh', 'British')
    if 'Fench' in naz:
        naz= naz.replace('Fench', 'French')
    if 'active in France by 1894' in naz:
        naz= naz.replace('active in France by 1894', 'French')
    if 'french' == naz:
        naz= naz.replace('french', 'French')
    if 'France' == naz:
        naz= naz.replace('France', 'French')
    if ' / ' in naz:
        naz= naz.replace(' / ', '-')
    if '/' in naz:
        naz= naz.replace('/', '-')
    if 'possibly ' in naz:
        naz= naz.replace('possibly ', '')
    if 'probably ' in naz:
        naz= naz.replace('probably ', '')
    if ' [search purposes only]' in naz:
        naz= naz.replace(' [search purposes only]', '')
    if 'U.S.' in naz:
        naz= naz.replace('U.S.', 'United States')
    if 'Nethrlandish' in naz:
        naz= naz.replace('Nethrlandish', 'Netherlandish')
    if 'Netherlands' in naz:
        naz= naz.replace('Netherlands', 'Netherlandish')
    if 'German-born sculptor later active in Switzerland' in naz:
        naz= naz.replace('German-born sculptor later active in Switzerland', 'German')
    if 'Hannover' in naz:
        naz= naz.replace('Hannover','German')
    if 'German-Solingen' in naz:
        naz= naz.replace('German-Solingen','German')
    if 'italian' in naz:
        naz= naz.replace('italian','Italian')
    if 'Italy' in naz:
        naz= naz.replace('Italy','Italian')
    if 'Dem. Republic of the Congo' == naz:
        naz= naz.replace('Dem. Republic of the Congo', 'Democratic Republic of Congo')
    if 'Hanoverian' == naz:
        naz= naz.replace('Hanoverian', 'German')
    if 'Germany' == naz:
        naz= naz.replace('Germany', 'German')
    if 'Finish' in naz:
        naz= naz.replace('Finish', 'Finnish')
    if 'Spainish' == naz:
        naz= naz.replace('Spainish', 'Spanish')
    if 'Central European' in naz:
        naz= naz.replace('Central European', 'European')
    if 'Chiricahua Apache Native American' in naz:
        naz= naz.replace('Chiricahua Apache Native American', 'Native American')
    if 'Algonquin family' in naz:
        naz= naz.replace('Algonquin family', 'Native American')
    if 'Continental-Swiss' in naz:
        naz= naz.replace('Continental-Swiss', 'Swiss')
    if 'Siena' in naz:
        naz= naz.replace('Siena', 'Sienese')
    if 'Italian French' in naz:
        naz= naz.replace('French Italian', 'Italian French')
    if 'Africa' in naz:
        naz= naz.replace('Africa', 'African')
    if 'Africann' in naz:
        naz= naz.replace('Africann', 'African')
    if 'Coast Africa' in naz:
        naz= naz.replace('Coast Africa', 'African')
    if 'New York' in naz:
        naz= naz.replace('New York', 'American')
    if 'Antelope Valley Washoe' in naz:
        naz= naz.replace('Antelope Valley Washoe', 'American')
    if 'Venice' in naz:
        naz= naz.replace('Venice', 'Venetian')
    if 'Dutch' in naz:
        naz= naz.replace('Dutch', 'Netherlandish')
    
    return naz 


In [46]:
MetNew = Met.copy(deep=True)
MetNew = MetNew[MetNew['Nationality'].notna()]
MetNew = MetNew[MetNew['Nationality'].str.contains(' or ')==False]
MetNew = MetNew[MetNew['Nationality'].str.contains('Manufacturer')==False]
MetNew = MetNew[MetNew['Nationality'].str.contains('Continental')==False]
MetNew["Nationality"] = MetNew["Nationality"].apply(cleanNazMet)
MetNew.to_csv("MetNew.csv")

In [47]:
museums=['MetNew.csv']
names = ['Met']
for museum in museums:
    with open(museum, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        nationalities = set() 
        for item in reader:
            if '|' in item['Nationality']:
                    item['Nationality'] = item['Nationality'].split('|')
                    for n in range(len(item['Nationality'])):
                        nationality= item['Nationality'][n].strip()
                        nationalities.add(nationality)
            else:
                nationalities.add(item['Nationality'].strip())
print(nationalities)

{'', 'Costa Rican', 'Greek', 'Jamaican', 'British', 'Egyptian', 'Franco-Flemish', 'Greenlandish', 'Kashmir', 'Sienese', 'Flemish', 'American England', 'Norway', 'Chinese', 'Asmat', 'Mexican', 'Nigeria', 'Norwegian', 'Netherlandish-French', 'Guatemalan', 'English', 'Uganda', 'Aachen', 'Hanoverian', 'England', 'Byzantine', 'Malaysian', 'Democratic Republic of Congo', 'Slovakian', 'Venetian', 'Franco-American', 'Armenian-Iranian', 'Romanian', 'British French', 'Italian French', 'Africann', 'Argentinian', 'Native American', 'French American', 'Samoan', 'Netherlandish', 'American German', 'Saint Lucian', 'Congo', 'Scottish', 'New Zealander', 'Afghani', 'French-English', 'Portuguese', 'Sri Lankan', 'French British', 'Serbo-croat', 'Iranian-American', 'American-Canadian', 'Venezuelan', 'Spanish', 'Albanian', 'United States', 'Manchu', 'Malian', 'American Britain', 'Florence', 'European', 'Mexican-American', 'Panamanian', 'Scandinavian', 'French-Netherlandish', 'Colombian', 'Persian', 'Polish'

In [52]:
with open('MetNew.csv', mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        MetNationalities ={}
        for nationality in nationalities:  
            MetNationalities[nationality]= 0
        for item in reader:
        
            for naz in nationalities:
                    if naz in item['Nationality']:
                        MetNationalities[naz] += 1 
print(MetNationalities)

{'': 275417, 'Costa Rican': 1, 'Greek': 170, 'Jamaican': 1, 'British': 18794, 'Egyptian': 10, 'Franco-Flemish': 1, 'Greenlandish': 1, 'Kashmir': 1, 'Sienese': 5, 'Flemish': 1560, 'American England': 1, 'Norway': 1, 'Chinese': 2058, 'Asmat': 2, 'Mexican': 358, 'Nigeria': 22, 'Norwegian': 85, 'Netherlandish-French': 3, 'Guatemalan': 1, 'English': 9, 'Uganda': 1, 'Aachen': 1, 'Hanoverian': 1, 'England': 3, 'Byzantine': 1, 'Malaysian': 31, 'Democratic Republic of Congo': 2, 'Slovakian': 1, 'Venetian': 4, 'Franco-American': 1, 'Armenian-Iranian': 2, 'Romanian': 13, 'British French': 1, 'Italian French': 2, 'Africann': 1, 'Argentinian': 19, 'Native American': 6, 'French American': 1, 'Samoan': 5, 'Netherlandish': 9297, 'American German': 9, 'Saint Lucian': 5, 'Congo': 5, 'Scottish': 412, 'New Zealander': 5, 'Afghani': 1, 'French-English': 2, 'Portuguese': 33, 'Sri Lankan': 1, 'French British': 1, 'Serbo-croat': 1, 'Iranian-American': 1, 'American-Canadian': 3, 'Venezuelan': 15, 'Spanish': 24

# Nationalities at Nga

In [49]:
NgaNationalities = pd.read_csv('https://raw.githubusercontent.com/NationalGalleryOfArt/opendata/main/data/constituents.csv')
pd.set_option('display.max_columns', None)
NgaNationalities = NgaNationalities[['artistofngaobject', 'nationality']]
NgaNationalities = NgaNationalities[NgaNationalities['artistofngaobject']==1]
NgaNationalities = NgaNationalities[NgaNationalities['nationality'].notna()]
NgaNationalities = NgaNationalities[NgaNationalities['nationality'] != 'Unknown']
NgaNationalities = NgaNationalities [NgaNationalities ['nationality'].str.contains(' or ')==False]



In [50]:
def cleanNazNga(naz):
    if ' (?)' in naz:
         naz = naz.replace(' (?)', '')
    if 'italian' in naz:
         naz = naz.replace('italian', 'Italian')
    if '/' in naz:
         naz = naz.replace('/', '-')
    if '\u206eItalian' in naz:
         naz = naz.replace('\u206eItalian', 'Italian')
    if ' and ' in naz:
        naz = naz.replace(' and ', '-')
        naz= naz.split('-')
        naz= naz[1] + '-' + naz[0]
    if 'South ' in naz:
        naz= naz.replace('South ', '')
    if 'West ' in naz:
        naz= naz.replace('West ', '')
        
      
    
    return naz   

In [169]:
NgaNationalities["nationality"] = NgaNationalities["nationality"].apply(cleanNazNga)
NgaNationalities.to_csv("NgaNationalities.csv")

In [267]:
with open('NgaNationalities.csv', mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        nationalities = set() 
        for item in reader:
            nationality= item['nationality'].strip()
            nationalities.add(nationality)

print(nationalities)

{'British', 'Welsh', 'Franco-Flemish', 'Dalmatian', 'Croatian', 'New Zealander', 'Flemish', 'Dutch', 'Cuban', 'Bosnian', 'Iranian', 'Swiss', 'Spanish', 'Albanian', 'Slovenian', 'Bohemian', 'Argentinean', 'Irish', 'Mexican-American', 'American', 'Persian', 'Slovak', 'Scottish', 'Czech', 'French', 'Netherlandish', 'Argentine', 'Chilean', 'Germany', 'Romanian', 'Greek', 'Yugoslavian', 'Egyptian', 'Scandinavian', 'Belgian', 'Latvian', 'Australian', 'Armenian', 'Hispano-Flemish', 'Genoese', 'Swedish', 'Uruguayan', 'Japanese', 'Mexican', 'Chinese', 'Brazilian', 'Lithuanian', 'Moroccan', 'Bolognese', 'Etruscan', 'Korean', 'Hungarian', 'German', 'New Zealand', 'Turkish', 'Israeli', 'Malian', 'Danish', 'Colombian', 'Nigerian', 'Roman', 'Czech-Israeli', 'Guatemalan', 'Indonesian', 'Austrian', 'Portuguese', 'Peruvian', 'Mosan', 'Italian', 'Nicaraguan', 'African', 'Russian', 'Polish', 'Bavarian', 'Icelandic', 'Veneto-Islamic', 'Canadian', 'English', 'Indian', 'Ukrainian', 'Byzantine', 'Liechtenste

In [268]:
with open('NgaNationalities.csv', mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        NgaNationalities ={}
        for nationality in nationalities:  
            NgaNationalities[nationality]= 0
        for item in reader:
            for naz in nationalities:
                    if naz == item['nationality']:
                        NgaNationalities[naz] += 1 
print(NgaNationalities)

{'British': 1144, 'Welsh': 10, 'Franco-Flemish': 3, 'Dalmatian': 1, 'Croatian': 2, 'New Zealander': 2, 'Flemish': 251, 'Dutch': 472, 'Cuban': 8, 'Bosnian': 1, 'Iranian': 2, 'Swiss': 157, 'Spanish': 92, 'Albanian': 1, 'Slovenian': 1, 'Bohemian': 11, 'Argentinean': 20, 'Irish': 29, 'Mexican-American': 2, 'American': 6807, 'Persian': 3, 'Slovak': 1, 'Scottish': 55, 'Czech': 177, 'French': 2176, 'Netherlandish': 111, 'Argentine': 2, 'Chilean': 6, 'Germany': 1, 'Romanian': 6, 'Greek': 23, 'Yugoslavian': 14, 'Egyptian': 2, 'Scandinavian': 2, 'Belgian': 52, 'Latvian': 2, 'Australian': 22, 'Armenian': 1, 'Hispano-Flemish': 1, 'Genoese': 1, 'Swedish': 39, 'Uruguayan': 2, 'Japanese': 104, 'Mexican': 37, 'Chinese': 22, 'Brazilian': 16, 'Lithuanian': 2, 'Moroccan': 1, 'Bolognese': 1, 'Etruscan': 1, 'Korean': 5, 'Hungarian': 27, 'German': 1205, 'New Zealand': 1, 'Turkish': 2, 'Israeli': 19, 'Malian': 1, 'Danish': 26, 'Colombian': 2, 'Nigerian': 2, 'Roman': 9, 'Czech-Israeli': 1, 'Guatemalan': 2, 'I

# Nationalities at Tate

In [4]:
TateArtists = pd.read_csv('https://raw.githubusercontent.com/tategallery/collection/master/artist_data.csv')
TateArtists = TateArtists[TateArtists['placeOfBirth'].notna()]

In [89]:
def cleanNazTate(naz):
    if ',' in naz:
        naz = naz.split(',')[1]
    if naz == 'Blackheath':
        naz= naz.replace('Blackheath', 'United Kingdom')
    if naz == 'London':
        naz= naz.replace('London', 'United Kingdom')
    if naz == 'Kensington':
        naz= naz.replace('Kensington', 'United Kingdom')
    if naz == 'Chung-hua Min-kuo':
        naz= naz.replace('Chung-hua Min-kuo', 'Taiwan')
    if naz == 'Solothurn':
        naz= naz.replace('Solothurn', 'Schweiz')
    if naz == 'Melmerby':
        naz= naz.replace('Melmerby', 'United Kingdom')
    if naz == 'Montserrat':
        naz= naz.replace('Montserrat', 'España')
    if naz == 'Canterbury':
        naz= naz.replace('Canterbury', 'United Kingdom')
    if naz == 'Staten Island':
        naz= naz.replace('Staten Island', 'United States')
    if naz == 'Epsom':
        naz= naz.replace('Epsom', 'United Kingdom')
    if naz == 'Wimbledon':
        naz= naz.replace('Staten Island', 'United Kingdom')
    if naz == 'Plymouth':
        naz= naz.replace('Plymouth', 'United Kingdom')
    if naz == 'Wimbledon':
        naz= naz.replace('Wimbledon', 'United Kingdom')
    if naz == 'Edinburgh':
        naz= naz.replace('Edinburgh', 'United Kingdom')
    if naz == 'Beckington':
        naz= naz.replace('Beckington', 'United Kingdom')
    if naz == 'Hertfordshire':
        naz= naz.replace('Hertfordshire', 'United Kingdom')
    if naz == 'Isle of Man':
        naz= naz.replace('Isle of Man', 'United Kingdom')
    if naz == 'Bristol':
        naz= naz.replace('Bristol', 'United Kingdom')
    if naz == 'Liverpool':
        naz= naz.replace('Liverpool', 'United Kingdom')
    if naz == 'Braintree':
        naz= naz.replace('Braintree', 'United Kingdom')
    if naz == 'Stoke on Trent':
        naz= naz.replace('Stoke on Trent', 'United Kingdom')
    if naz == 'Rochdale':
        naz= naz.replace('Rochdale', 'United Kingdom')
    if 'D.C.' in naz:
        naz= naz.replace('D.C.', 'Colombia')
    if 'Otok' in naz:
        naz= naz.replace('Otok', 'Hrvatska')
    if 'Département de la' in naz:
        naz= naz.replace('Département de la', 'France')
    if naz == 'Niederschlesien':
        naz= naz.replace('Niederschlesien', 'Polska')
    if naz == 'Perth':
        naz= naz.replace('Perth', 'Australia')
    if naz == 'Bermondsey':
        naz= naz.replace('Bermondsey', 'United Kingdom')
    if naz == 'Egremont':
        naz= naz.replace('Egremont', 'United Kingdom')
    if naz == 'Charlotte Amalie':
        naz= naz.replace('Charlotte Amalie', 'United States')
    if naz == 'Charlieu':
        naz= naz.replace('Charlieu', 'France')
    if naz == 'Stockholm':
        naz= naz.replace('Stockholm', 'Sverige')
    if naz == 'Auteuil':
        naz= naz.replace('Auteuil', 'France')
   
    return naz   

In [90]:
TateArtists["placeOfBirth"] = TateArtists["placeOfBirth"].apply(cleanNazTate)
TateArtists.to_csv("TateArtists.csv")

In [270]:
with open('TateArtists.csv', mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        nationalities = set() 
        for item in reader:
           
            nationality= item['placeOfBirth'].strip()
            nationalities.add(nationality)

print(nationalities)

{'Hrvatska', 'Italia', 'Ceská Republika', 'Belarus', 'België', 'As-Sudan', 'Ísland', 'Taiwan', 'Pakistan', 'Singapore', 'Éire', 'Portugal', 'Bosna i Hercegovina', 'Saint Hélier', 'Shqipëria', 'Myanmar', 'Nihon', 'Perú', 'South Africa', 'Zhonghua', 'România', 'Canada', 'Polska', 'Bangladesh', 'Suomi', 'Al-‘Iraq', 'Mehoz', 'España', 'Jugoslavija', 'Suriyah', 'Eesti', "Yisra'el", 'Al-Lubnan', 'Zambia', 'Danmark', 'Schweiz', 'Lietuva', 'Slovenija', 'Tunis', 'Ukrayina', 'Venezuela', 'Uganda', 'Nicaragua', 'Magyarország', 'Misr', 'Schlesien', 'Indonesia', 'México', 'Chile', 'Malta', 'Ellás', 'Sri Lanka', 'Prathet Thai', "Taehan Min'guk", 'Samoa', 'Lao', 'New Zealand', 'Slovenská Republika', 'Bénin', 'Malaysia', 'Barbados', 'Mauritius', 'France', 'Cameroun', 'Nigeria', 'Australia', 'Douglas', 'Argentina', 'Kenya', 'Nederland', 'Viet Nam', 'Tanzania', 'Türkiye', 'Moldova', 'Pilipinas', 'Bharat', 'United States', 'Armenia', 'Bahamas', 'Latvija', 'Cuba', 'Makedonija', 'Jamaica', 'Brasil', 'Bulga

In [274]:
with open('TateArtists.csv', mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        TateNationalities ={}
        for nationality in nationalities:  
            TateNationalities[nationality]= 0
        for item in reader:
            for naz in nationalities:
                    if naz in item['placeOfBirth']:
                        TateNationalities[naz] += 1 
print(TateNationalities)

{'Hrvatska': 9, 'Italia': 80, 'Ceská Republika': 14, 'Belarus': 4, 'België': 37, 'As-Sudan': 1, 'Ísland': 1, 'Taiwan': 1, 'Pakistan': 5, 'Singapore': 2, 'Éire': 51, 'Portugal': 10, 'Bosna i Hercegovina': 2, 'Saint Hélier': 2, 'Shqipëria': 1, 'Myanmar': 1, 'Nihon': 27, 'Perú': 4, 'South Africa': 20, 'Zhonghua': 22, 'România': 13, 'Canada': 40, 'Polska': 42, 'Bangladesh': 2, 'Suomi': 1, 'Al-‘Iraq': 1, 'Mehoz': 2, 'España': 30, 'Jugoslavija': 3, 'Suriyah': 2, 'Eesti': 1, "Yisra'el": 11, 'Al-Lubnan': 8, 'Zambia': 1, 'Danmark': 9, 'Schweiz': 30, 'Lietuva': 4, 'Slovenija': 6, 'Tunis': 1, 'Ukrayina': 17, 'Venezuela': 7, 'Uganda': 1, 'Nicaragua': 1, 'Magyarország': 13, 'Misr': 8, 'Schlesien': 2, 'Indonesia': 2, 'México': 13, 'Chile': 5, 'Malta': 1, 'Ellás': 10, 'Sri Lanka': 3, 'Prathet Thai': 1, "Taehan Min'guk": 3, 'Samoa': 1, 'Lao': 1, 'New Zealand': 10, 'Slovenská Republika': 3, 'Bénin': 1, 'Malaysia': 1, 'Barbados': 1, 'Mauritius': 2, 'France': 160, 'Cameroun': 1, 'Nigeria': 1, 'Australia'

#Our analysis.
<br>
1. What are the most acquired artists in museums (in general)?
    - Is there a gender gap in the selection of artists? UOMO-DONNA BASIC
    - What are the most represented nationalities (in general)? A TORTA/MENO BASIC MA IL CONCETTO E' QUELLO
    - What are the most represented movements or genres (in general)? STESSO COME SOPRA
2. How have acquisition criteria changed (over time) in museums?
    - In which years are artists' works mostly acquired? BAR CHART/RADIAL BARCHART PER OGNI MUSEO + LINEPLOT PER COMPARARE I 4
    - When does the gender gap decreases (if it does)? 
        COMPUTE PERCENTAGE OF MALE/FEMALE ARTIST ACQUIRED IN TIMESLOTS + COMPARE ACROSS TIME SLOTS 
        PIECHART/ STACKED BAR CHART WITH ONLY 2 COLORS
    - In which years artists' nationalities more influent on the selection? STACKED BAR CHART, SEE TELEGRAM INSPO
    - In which years artists' movements/genres more influent on the selection?
3. If we compare criteria of all museums, in general and over time, do we see any similarity or significant difference?
    - Do certain museums acquire more works based on artists/artists' gender/nationality/movement than others?

Acquisition criteria.
1.  In which years are artists' works mostly acquired?<br>
To answer, we need to count how many times each year shows up in the DateAcquired column.

In [ ]:
MoMa['year'] = pd.DatetimeIndex(MoMa['DateAcquired']).year
MoMaNew['DateAcquired'] = MoMa['year']
MoMaNew = MoMaNew[MoMaNew['DateAcquired'].notna()]
MoMaNew.to_csv("MoMaNew.csv")

In [60]:
with open('Tate.csv', mode='r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    years={}
    for item in reader:
        if item['DateAcquired']not in years:
            years[item['DateAcquired']]= 1
        else:
            years[item['DateAcquired']]+= 1

    print(years)
    #all_years=list(years.keys())
    #print(sorted(all_years))
    

{'1922': 115, '1919': 127, '1896': 37, '1916': 37, '1924': 226, '1925': 227, '1927': 225, '1912': 81, '1910': 86, '1909': 50, '1926': 71, '1888': 1060, '1847': 148, '1900': 74, '1940': 207, '1908': 123, '1830': 2, '1877': 11, '1949': 45, '1867': 15, '1905': 64, '1891': 21, '1856': 37893, '1961': 77, '1941': 104, '1957': 51, '1960': 81, '1939': 79, '1946': 128, '1975': 3046, '2008': 787, '2009': 1364, '2010': 392, '2011': 320, '2012': 514, '2013': 458, '1837': 4, '1826': 4, '1823': 1, '1824': 2, '1828': 3, '1835': 1, '1827': 1, '1836': 3, '1841': 2, '1842': 1, '1843': 1, '1849': 1, '1853': 3, '1854': 17, '1857': 2, '1858': 4, '1859': 17, '1860': 2, '1861': 4, '1862': 2, '1863': 1, '1864': 2, '1866': 2, '1876': 3, '1868': 10, '1869': 5, '1870': 1, '1871': 9, '1872': 3, '1873': 1, '1874': 3, '1875': 3, '1834': 1, '1840': 2, '1878': 15, '1879': 49, '1880': 6, '1882': 6, '1883': 3, '1884': 9, '1885': 22, '1886': 9, '1887': 12, '1889': 9, '1890': 8, '1892': 14, '1923': 39, '1893': 9, '1894':

In [61]:
new_dict={}
for key in years:
    key_int=key.split('.')[0]
    key_int=int(key_int)
    if key_int in range(1928,1941):
        if '1930s' not in new_dict.keys():
               new_dict['1930s']= years[key]
        else:
            new_dict['1930s'] += years[key]
    if key_int in range(1940,1951):
        if '1940s' not in new_dict.keys():
               new_dict['1940s']= years[key]
        else:
            new_dict['1940s'] += years[key]
    
    if key_int in range(1950,1961):
        if '1950s' not in new_dict.keys():
               new_dict['1950s']= years[key]
        else:
            new_dict['1950s'] += years[key]
    
    if key_int in range(1960,1971):
        if '1960s' not in new_dict.keys():
               new_dict['1960s']= years[key]
        else:
            new_dict['1960s'] += years[key]
    
    if key_int in range(1970,1981):
        if '1970s' not in new_dict.keys():
               new_dict['1970s']= years[key]
        else:
            new_dict['1970s'] += years[key]
    if key_int in range(1980,1991):
        if '1980s' not in new_dict.keys():
               new_dict['1980s']= years[key]
        else:
            new_dict['1980s'] += years[key]
    
    if key_int in range(1990,2001):
        if '1990s' not in new_dict.keys():
               new_dict['1990s']= years[key]
        else:
            new_dict['1990s'] += years[key]
        
    
print(new_dict)
    

{'1930s': 788, '1940s': 776, '1960s': 1048, '1950s': 633, '1970s': 6853, '1980s': 5538, '1990s': 6693}


When does the gender gap decreases (if it does)? <br>
Per ogni 10 anni, percentuale di uomini e donne acquisiti e differenza.

In [ ]:
with open('MoMaNew.csv', mode='r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    years={}
    for item in reader:
        if item['DateAcquired']not in years:
            years[item['DateAcquired']]= 1
        else:
            years[item['DateAcquired']]+= 1

    print(years)

When does the gender gap decreases (if it does)?
Per ogni 10 anni, percentuale di uomini e donne acquisiti e differenza.

In which years artists' nationalities more influent on the selection?
Per ogni 10 anni, percentuale di nazionalità acquisite e differenza.

In which years artists'movements/genres more influent on the selection?
Per ogni 10 anni, percentuale di nazionalità acquisite e differenza.

Nga non ha Gender
Met molti Gender sono NaN


In [14]:
def cleanWikiIDs(id):
    if "|" in id:
        id = id.split("|")[0]
    id = id.split("wiki/")[-1]
    id = 'wd:'+id
    return id

In [41]:
MetCleanIds = Met.drop_duplicates(subset='Artist', keep="first")
MetCleanIds = MetCleanIds[MetCleanIds["Wiki QID"] != "|"]
MetCleanIds = MetCleanIds[MetCleanIds["Wiki QID"]!= "None"]
MetCleanIds = MetCleanIds[MetCleanIds["Wiki QID"]!= ""]
MetCleanIds["Wiki QID"] = MetCleanIds["Wiki QID"].apply(cleanWikiIDs)

46139

In [51]:
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl
import rdflib

ssl._create_default_https_context = ssl._create_unverified_context

# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


def searchWiki(flag, frame):
    artistsIds = frame["Wiki QID"].values
    upTo = 500+flag
    artistsSlot = artistsIds[flag:upTo]
    artists = ' '.join(artistsSlot) 

    artists_genders_from_ids = """
    SELECT DISTINCT ?artist ?gender ?genderLabel
    WHERE {{
      VALUES ?artist {"""+artists+"""} .
      ?artist wdt:P21 ?gender . 
      ?gender rdfs:label ?genderLabel .
        FILTER (langMatches(lang(?genderLabel), "EN"))
    }}
    """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(artists_genders_from_ids)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    genderDict={}

    # manipulate the result
    for result in results["results"]["bindings"]:
        artists_id =  'wd:'+result["artist"]["value"].split('/')[-1]
        if "genderLabel" in result: 
            gender_label = result["genderLabel"]["value"]
            genderDict[artists_id] = gender_label
        else:
            genderDict[artists_id] = None
    for index, row in frame.iterrows():
        for artist_id in genderDict:
                if row['Wiki QID']== artist_id:
                    frame.at[index,'Gender'] = genderDict[artist_id]
    return frame


In [53]:
#finire qui con timer del numero di rows diviso 500, così che si ripeta
ieration_num = 500
for iter in range(ieration_num):
    searchWiki(iter+500, MetCleanIds)

KeyboardInterrupt: 